In [ ]:
# from tensorflow.python.keras import models
# from tensorflow.keras import layers
# from tensorflow.python.keras import losses
# from tensorflow.keras import optimizers
# from tensorflow.python.keras import activations
# from tensorflow.python.keras import callbacks
# from tensorflow.python.keras import initializers
# from tensorflow.python.keras import backend
# from tensorflow.python.keras import metrics
# from tensorflow.python import keras
# import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
train_x_other = np.load('../input/ventilator-traindata/train_x_other.npy')
print(train_x_other.shape)


test_ori = pd.read_csv('../data/test.csv', index_col=None, header=0)


def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()

    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()

    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    # df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    # df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    # df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    # df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    # df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    # df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    # df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    # df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    # df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    # df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    # df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    # df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    # df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    # df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    # df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df = df.fillna(0)

    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    # df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')

    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    # df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df.drop(['u_in_lag1'], axis=1)
    # df.drop(['u_out_lag1'], axis=1)
    # df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    # df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']

    # df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    # df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']

    # df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    # df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    # df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    # df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross'] = df['u_in'] * df['u_out']
    df['cross2'] = df['time_step'] * df['u_out']

    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    return df


test = add_features(test_ori)
test.drop(['id', 'breath_id'], axis=1, inplace=True)
test = test.values
test_x_rc = test[:, -15:]
test_x_other = test[:, :-15]
scale = StandardScaler()
scale.fit(train_x_other)
test_x_other = scale.transform(test_x_other)

In [ ]:
np.save('../data/test_x_rc.npy', test_x_rc)
np.save('../data/test_x_other.npy', test_x_other)